In [8]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import pickle
from collections import Counter

# tokenizer
from konlpy.tag import Mecab

# torch
import torch
import torch.nn as nn
import torchtext
from torchtext import transforms as T
from torchtext.vocab import vocab

In [6]:
train = pd.read_csv('/root/share/data/train.csv')
test = pd.read_csv('/root/share/data/test.csv')
print(len(train))
print(len(test))

40000
5000


In [7]:
# 필요 단어만 남기기
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train = train[train['data'].notnull()]

test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']
# 조사 및 불필요 단어 제거 
def tokenizer(sentence, tag):
    return tag.morphs(sentence)
mecab = Mecab()
train['morph'] = train['data'].apply(tokenizer, tag=mecab)
train['morph'] = train['morph'].apply(lambda xs: [x for x in xs if x not in stop_words])

test['morph'] = test['data'].apply(tokenizer, tag=mecab)
test['morph'] = test['morph'].apply(lambda xs: [x for x in xs if x not in stop_words])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [9]:
# 단어 사전 만들기 
counter = Counter()
for sentence in train['morph']:
    counter.update(sentence)

vocab_f = vocab(counter, min_freq=5, specials=['<unk>','<pad>'])
vocab_f.set_default_index(vocab_f['<unk>'])

In [10]:
# text to num transform
text_transform = T.Sequential(
    T.VocabTransform(vocab_f),
    T.Truncate(max_seq_len=300),
    T.ToTensor(padding_value=vocab_f['<pad>'])
)

preprocess_train_output = text_transform(train['morph'].tolist())
preprocess_test_output = text_transform(test['morph'].tolist())

label = train.category.tolist()

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1138: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


In [13]:
# 데이터 저장
with open('/root/share/data/vocab.pkl','wb') as f:
    pickle.dump(vocab_f, f)

with open('/root/share/data/train_data.pkl','wb') as f:
    pickle.dump(preprocess_train_output, f)

with open('/root/share/data/train_label.pkl', 'wb') as f:
    pickle.dump(label, f)

with open('/root/share/data/test_data.pkl','wb') as f:
    pickle.dump(preprocess_test_output, f)